<a href="https://colab.research.google.com/github/al2501/fooddesert/blob/main/Geofood_Final_with_Labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Environment

In [ ]:
#Install newest branch
!pip install pysal

#Install the geopandas module
!pip install geopandas

In [ ]:
!pip install descartes
!pip install mapclassify

In [ ]:
from pysal import *
import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import pylab
import descartes
import mapclassify

%matplotlib inline

plt.style.use('ggplot')
pylab.rcParams['figure.figsize'] = (20., 16.)

## Import and Clean Local Authorities Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# importing and visualisating our shapefile for the first time

data_path = "/content/drive/MyDrive/QM data/third map geofood/Local_Authority_Districts_(December_2020)_UK_BFC.shp"

df1 = gp.read_file(data_path)

df1.tail()

,OBJECTID,LAD20CD,LAD20NM,LAD20NMW,BNG_E,BNG_N,LONG,LAT,Shape__Are,Shape__Len,geometry
374,375,W06000020,Torfaen,Torfaen,327459,200480,-3.05101,51.698360,1.262399e+08,82544.775967,"POLYGON ((323898.201 211287.499, 324115.698 21..."
375,376,W06000021,Monmouthshire,Sir Fynwy,337812,209231,-2.90280,51.778271,8.503266e+08,224917.251559,"MULTIPOLYGON (((327830.799 231005.105, 327840...."
376,377,W06000022,Newport,Casnewydd,337897,187432,-2.89769,51.582310,1.904311e+08,153216.416923,"MULTIPOLYGON (((342211.900 194845.495, 342221...."
377,378,W06000023,Powys,Powys,302329,273255,-3.43531,52.348640,5.195311e+09,610132.027463,"POLYGON ((322392.901 334017.198, 322378.002 33..."
378,379,W06000024,Merthyr Tydfil,Merthyr Tudful,305916,206424,-3.36425,51.748581,1.119570e+08,66696.929592,"POLYGON ((303435.502 214925.598, 303431.602 21..."


In [ ]:
df1['geometry'].head()

0    MULTIPOLYGON (((447213.899 537036.104, 447228....
1    MULTIPOLYGON (((448609.900 521982.600, 448616....
2    MULTIPOLYGON (((455932.335 527880.697, 455919....
3    MULTIPOLYGON (((444157.002 527956.304, 444165....
4    POLYGON ((423496.602 524724.299, 423497.204 52...
Name: geometry, dtype: geometry

In [ ]:
#cleaning the dataset
clean = df1[['LAD20CD','LAD20NM','geometry']]
UK_LTLAs=clean.replace(',', '', regex=True)


In [ ]:
UK_LTLAs.head()

,LAD20CD,LAD20NM,geometry
0,E06000001,Hartlepool,"MULTIPOLYGON (((447213.899 537036.104, 447228...."
1,E06000002,Middlesbrough,"MULTIPOLYGON (((448609.900 521982.600, 448616...."
2,E06000003,Redcar and Cleveland,"MULTIPOLYGON (((455932.335 527880.697, 455919...."
3,E06000004,Stockton-on-Tees,"MULTIPOLYGON (((444157.002 527956.304, 444165...."
4,E06000005,Darlington,"POLYGON ((423496.602 524724.299, 423497.204 52..."


In [ ]:
UK_LTLAs['geometry'].head()

0    MULTIPOLYGON (((447213.899 537036.104, 447228....
1    MULTIPOLYGON (((448609.900 521982.600, 448616....
2    MULTIPOLYGON (((455932.335 527880.697, 455919....
3    MULTIPOLYGON (((444157.002 527956.304, 444165....
4    POLYGON ((423496.602 524724.299, 423497.204 52...
Name: geometry, dtype: geometry

In [ ]:
UK_LTLAs.crs

<Projected CRS: EPSG:27700>
Name: OSGB36 / British National Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.0, 49.75, 2.01, 61.01)
Coordinate Operation:
- name: British National Grid
- method: Transverse Mercator
Datum: Ordnance Survey of Great Britain 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich

In [ ]:
UK_LTLAs.crs.name
UK_LTLAs.crs.datum

DATUM["Ordnance Survey of Great Britain 1936",
    ELLIPSOID["Airy 1830",6377563.396,299.3249646,
        LENGTHUNIT["metre",1]],
    ID["EPSG",6277]]

In [ ]:
type(UK_LTLAs.crs)

pyproj.crs.crs.CRS

In [ ]:
UK_LTLAs.head()

,LAD20CD,LAD20NM,geometry
0,E06000001,Hartlepool,"MULTIPOLYGON (((447213.899 537036.104, 447228...."
1,E06000002,Middlesbrough,"MULTIPOLYGON (((448609.900 521982.600, 448616...."
2,E06000003,Redcar and Cleveland,"MULTIPOLYGON (((455932.335 527880.697, 455919...."
3,E06000004,Stockton-on-Tees,"MULTIPOLYGON (((444157.002 527956.304, 444165...."
4,E06000005,Darlington,"POLYGON ((423496.602 524724.299, 423497.204 52..."


In [ ]:
UK_LTLAs.crs = 'epsg:27700'
target_crs = {'datum':'OSGB36', 'no_defs':True, 'proj':'merc'}
projected_UK_LTLAs = UK_LTLAs.to_crs(crs=target_crs)

## Import and Clean E-Food Desert Index (EFDI) Dataset

Retrieved from https://data.cdrc.ac.uk/dataset/e-food-desert-index/resource/efdi-england-and-wales

In [ ]:
#importing dataset 2
data_path = "/content/drive/MyDrive/QM data/third map geofood/efdi_england.csv"

food = pd.read_csv(data_path, encoding = 'latin1')
food.head()

,LSOA or DZ,Nation,Score,Rank,Decile
0,W01000532,Wales,70.82672,1.0,1.0
1,W01000545,Wales,70.54694,2.0,1.0
2,W01000471,Wales,70.51910,3.0,1.0
3,W01000529,Wales,70.37505,4.0,1.0
4,W01000506,Wales,70.23811,5.0,1.0


In [ ]:
#checking if there is no error
food.replace(',', '', regex=True, inplace=True)
food['Score'] = food['Score'].replace('-', 'NaN', regex=True).astype('float')
food['Score'].head()

0    70.82672
1    70.54694
2    70.51910
3    70.37505
4    70.23811
Name: Score, dtype: float64

In [ ]:
#cleaning the data
food = food.rename(columns={'LSOA or DZ':'LSOA'})
food.drop(labels=34753, axis=0)

,LSOA,Nation,Score,Rank,Decile
0,W01000532,Wales,70.82672,1.0,1.0
1,W01000545,Wales,70.54694,2.0,1.0
2,W01000471,Wales,70.51910,3.0,1.0
3,W01000529,Wales,70.37505,4.0,1.0
4,W01000506,Wales,70.23811,5.0,1.0
...,...,...,...,...,...
34748,W01001823,Wales,1.40386,34749.0,10.0
34749,W01001922,Wales,1.33212,34750.0,10.0
34750,E01014557,England,1.27383,34751.0,10.0
34751,E01014511,England,1.23816,34752.0,10.0


In [ ]:
food = food[food.Nation != 'Wales']
food.drop('Rank', axis=1, inplace=True)
food.drop('Nation', axis=1, inplace=True)
food.drop('Decile', axis=1, inplace=True)

In [ ]:
food.head()

,LSOA,Score
11,E01027390,68.40022
20,E01019259,66.39587
24,E01029014,65.32626
25,E01019305,65.22630
29,E01020126,64.61247


## Converting LSOAs to Local Authorities

File retrieved from https://geoportal.statistics.gov.uk/datasets/lower-layer-super-output-area-2011-to-ward-2020-lookup-in-england-and-wales/explore

In [ ]:
#importing file
data_path = "/content/drive/MyDrive/QM data/first map/loastoward.csv"
convert = pd.read_csv(data_path, encoding = 'latin1')
convert.head()

,ï»¿FID,LSOA11CD,LSOA11NM,WD20CD,WD20NM,LAD20CD,LAD20NM
0,1,E01011949,Hartlepool 009A,E05008945,Foggy Furze,E06000001,Hartlepool
1,2,E01012162,Redcar and Cleveland 012A,E05012454,Skelton East,E06000003,Redcar and Cleveland
2,3,E01011950,Hartlepool 008A,E05008942,Burn Valley,E06000001,Hartlepool
3,4,E01012163,Redcar and Cleveland 012B,E05012454,Skelton East,E06000003,Redcar and Cleveland
4,5,E01012164,Redcar and Cleveland 012C,E05012455,Skelton West,E06000003,Redcar and Cleveland


In [ ]:
# cleaning dataset by removing unecessary columns
convert.drop('WD20CD', axis=1, inplace=True)
convert.drop('WD20NM', axis=1, inplace=True)
convert.drop('LAD20NM', axis=1, inplace=True)
convert.drop('LSOA11NM', axis=1, inplace=True)
convert.drop('ï»¿FID', axis=1, inplace=True)

In [ ]:
convert = convert.rename(columns={'LSOA11CD':'LSOA'})

In [ ]:
convert.head()

,LSOA,LAD20CD
0,E01011949,E06000001
1,E01012162,E06000003
2,E01011950,E06000001
3,E01012163,E06000003
4,E01012164,E06000003


## Merging EFDI Dataset with Local Authorities Dataset

In [ ]:
merge1= convert.merge(food,right_on='LSOA',left_on='LSOA')
merge1.tail()

,LSOA,LAD20CD,Score
32839,E01033604,E09000033,18.54483
32840,E01033605,E09000033,32.57577
32841,E01033606,E09000033,8.01943
32842,E01033607,E09000033,9.51512
32843,E01033608,E09000033,9.77768


In [ ]:
merge1.drop(columns=["LSOA"],inplace=True)

In [ ]:
merge1.tail()

,LAD20CD,Score
32839,E09000033,18.54483
32840,E09000033,32.57577
32841,E09000033,8.01943
32842,E09000033,9.51512
32843,E09000033,9.77768


In [ ]:
df = pd.DataFrame(merge1, columns = ['LAD20CD', 'Score'])
df

,LAD20CD,Score
0,E06000001,33.41165
1,E06000003,21.06797
2,E06000001,42.01377
3,E06000003,13.95741
4,E06000003,23.39422
...,...,...
32839,E09000033,18.54483
32840,E09000033,32.57577
32841,E09000033,8.01943
32842,E09000033,9.51512


In [ ]:
new_val = df[df.duplicated('LAD20CD')]
print(new_val)

         LAD20CD     Score
2      E06000001  42.01377
3      E06000003  13.95741
4      E06000003  23.39422
5      E06000001  29.16406
6      E06000003  32.24583
...          ...       ...
32839  E09000033  18.54483
32840  E09000033  32.57577
32841  E09000033   8.01943
32842  E09000033   9.51512
32843  E09000033   9.77768

[32530 rows x 2 columns]


In [ ]:
mean1 = df.groupby('LAD20CD').mean({'Score':'first'}).reset_index()
print (mean1)

       LAD20CD      Score
0    E06000001  29.275454
1    E06000002  28.090475
2    E06000003  26.600593
3    E06000004  21.535062
4    E06000005  23.984222
..         ...        ...
309  E09000029  13.012150
310  E09000030  15.402752
311  E09000031  15.264457
312  E09000032  10.025566
313  E09000033  16.678192

[314 rows x 2 columns]


## Completing Final Merged Dataset

In [ ]:
geofood = mean1.merge(UK_LTLAs,right_on='LAD20CD',left_on='LAD20CD')
geofood.tail()

,LAD20CD,Score,LAD20NM,geometry
309,E09000029,13.012150,Sutton,"POLYGON ((527085.942 167617.691, 527087.728 16..."
310,E09000030,15.402752,Tower Hamlets,"POLYGON ((536776.386 184446.881, 536777.600 18..."
311,E09000031,15.264457,Waltham Forest,"POLYGON ((537921.098 196048.201, 537947.499 19..."
312,E09000032,10.025566,Wandsworth,"POLYGON ((530059.620 177871.478, 530066.940 17..."
313,E09000033,16.678192,Westminster,"POLYGON ((526755.127 183684.425, 526773.270 18..."


In [ ]:
geofood['Score']=geofood['Score'].round(decimals=3)
geofood['Score'].head()

0    29.275
1    28.090
2    26.601
3    21.535
4    23.984
Name: Score, dtype: float64

In [ ]:
geofood.to_csv('geofood_final.csv')
!cp geofood_final.csv "/content/drive/MyDrive/QM data/third map geofood"

In [ ]:
geofood = gp.GeoDataFrame(geofood, geometry='geometry')
geofood.to_file('geofood_final.zip', driver='ESRI Shapefile')
!cp geofood_final.zip "/content/drive/MyDrive/QM data/third map geofood"

cp: -r not specified; omitting directory 'geofood_final.zip'
